# 🏭 Intelligent System for Industrial Equipment Failure Diagnosis
## Interactive Dashboard

Use this dashboard to diagnose **Spindle Overheat** events in the CNC milling machine.

**Instructions:**
1. Adjust the **Sensor Readings** below (simulating live telemetry).
2. Click **Run Diagnosis** to trigger the Hybrid AI.
3. View the **Probabilistic Analysis** and **Action Recommendation**.

In [1]:
# === SYSTEM SETUP ===
import sys
import os
import warnings
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Add project root to path
sys.path.append(os.path.abspath(''))

# Import System Modules
from src.integration import run_real
from src.utils import load_cfg

# Suppress warnings for clean UI
warnings.filterwarnings('ignore')

# Load Config
cfg = load_cfg()
sensors = cfg['bn'].get('sensors', [])

print("✅ System Loaded. Ready for Diagnosis.")

✅ System Loaded. Ready for Diagnosis.


In [2]:
# === INTERFACE CONTROLS ===

style = {'description_width': 'initial'}

# Title
header = widgets.HTML("<h3>📡 Sensor Telemetry Input</h3>")

# Define Sensor Ranges (Derived from Telemetry Data)
# Format: (min, max, step)
sensor_config = {
    "spindle_temp": (20.0, 120.0, 0.1),
    "ambient_temp": (15.0, 40.0, 0.1),
    "vibration_rms": (0.0, 5.0, 0.01),
    "coolant_flow": (0.0, 10.0, 0.1),
    "feed_rate": (0.0, 5.0, 0.1),
    "spindle_speed": (0, 5000, 100), # Integer
    "load_pct": (0.0, 1.0, 0.01),
    "power_kw": (0.0, 10.0, 0.1),
    "tool_wear": (0.0, 0.5, 0.001)
}

sensor_widgets = {}
for s in sensors:
    if s in sensor_config:
        min_v, max_v, step = sensor_config[s]
        if isinstance(min_v, int) and isinstance(step, int):
            # Integer Slider for Speed etc.
            w = widgets.IntSlider(
                value=min_v,
                min=min_v,
                max=max_v,
                step=step,
                description=s.replace('_', ' ').title(),
                style=style,
                continuous_update=False
            )
        else:
            # Float Slider for others
            w = widgets.FloatSlider(
                value=min_v,
                min=min_v,
                max=max_v,
                step=step,
                description=s.replace('_', ' ').title(),
                style=style,
                continuous_update=False
            )
        sensor_widgets[s] = w
    else:
        # Fallback
        sensor_widgets[s] = widgets.FloatSlider(
            value=0.0, min=0.0, max=100.0, step=0.1,
            description=s.replace('_', ' ').title(),
            style=style
        )

# Layout inputs in a grid
left_box = widgets.VBox(list(sensor_widgets.values())[:len(sensors)//2])
right_box = widgets.VBox(list(sensor_widgets.values())[len(sensors)//2:])
ui_inputs = widgets.HBox([left_box, right_box])

# Action Buttons
btn_diagnose = widgets.Button(
    description='🔍 Run Diagnosis',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px'),
    icon='check'
)

btn_random = widgets.Button(
    description='🎲 Random Fault',
    button_style='warning',
    layout=widgets.Layout(width='150px'),
    icon='random'
)

output_area = widgets.Output()

# === LOGIC ===

def on_diagnose_click(b):
    with output_area:
        clear_output()
        # 1. Gather Evidence
        evidence = {s: w.value for s, w in sensor_widgets.items()}
        
        print("Running Hybrid Inference... Please wait.")
        
        try:
            # 2. Run Pipeline
            # Logic in run_real auto-discretizes floats -> bins
            result = run_real(evidence=evidence, debug=False, force_retrain=False)
            
            # 3. Render Results Pretty
            clear_output()
            render_dashboard(result, evidence)
            
        except Exception as e:
            print(f"❌ Error during diagnosis: {e}")
            print("Tip: Ensure 'python main.py' (Real Mode) has been run once to train the model.")

def on_random_click(b):
    # Set random values within defined ranges
    import random
    for s, w in sensor_widgets.items():
        if s in sensor_config:
            min_v, max_v, step = sensor_config[s]
            if isinstance(w, widgets.IntSlider):
                w.value = random.randint(int(min_v), int(max_v))
            else:
                w.value = round(random.uniform(min_v, max_v), 2)
    
    # Trigger diagnosis
    on_diagnose_click(b)

def render_dashboard(res, evidence):
    p_oh = res['p_overheat']
    cause = res['top_cause']
    action = res['recommended_action']
    cost = sum(res['expected_costs'].values()) if res.get('expected_costs') else 0
    
    # Color coding
    color = "green"
    status = "NORMAL"
    if p_oh > 0.3: 
        color = "orange"
        status = "WARNING"
    if p_oh > 0.7:
        color = "red"
        status = "CRITICAL"
        
    html = f"""
    <div style='border: 2px solid {color}; border-radius: 10px; padding: 20px; background-color: #f9f9f9;'>
        <h2 style='color: {color}; margin-top: 0;'>SYSTEM STATUS: {status}</h2>
        
        <div style='display: flex; justify-content: space-between;'>
            <div style='width: 45%;'>
                <h4>🔍 Bayesian Diagnosis</h4>
                <ul>
                    <li><b>Overheat Probability:</b> {p_oh:.1%}</li>
                    <li><b>Most Likely Cause:</b> <code>{cause}</code></li>
                </ul>
                
                <h5>Observed Symptoms (Evidence):</h5>
                <small>{evidence}</small>
            </div>
            
            <div style='width: 45%; border-left: 1px solid #ddd; padding-left: 20px;'>
                <h4>🛠️ Recommended Action</h4>
                <div style='font-size: 1.2em; font-weight: bold; color: #333;'>
                    ➤ {action}
                </div>
                <br>
                <table style='width:100%; font-size: 0.9em;'>
                    <tr><td><b>Components:</b></td><td>{', '.join(res.get('components', []))}</td></tr>
                    <tr><td><b>Procedures:</b></td><td>{', '.join(res.get('procedures', []))}</td></tr>
                </table>
            </div>
        </div>
    </div>
    """
    display(HTML(html))
    
    if 'probabilities' in res:
        display(HTML("<h4>📊 Root Cause Probability Distribution</h4>"))
        for c, p in res['probabilities'].items():
            bar_width = int(p * 300)
            display(HTML(f"<div style='margin-bottom: 5px;'>{c}: <b>{p:.1%}</b><div style='background-color: #4CAF50; width: {bar_width}px; height: 10px; border-radius: 5px;'></div></div>"))

btn_diagnose.on_click(on_diagnose_click)
btn_random.on_click(on_random_click)

display(header, ui_inputs, widgets.HBox([btn_diagnose, btn_random]), output_area)

HTML(value='<h3>📡 Sensor Telemetry Input</h3>')

Output()